In [7]:
using LinearAlgebra
using HierarchicalDA
using PDMats
using LinearMaps
using SparseArrays
using UnPack
import Base: *, size
using IterativeSolvers
using Preconditioners

In [2]:
Nx = 100
Ny = 50
Ns = 80

80

In [3]:
H = LinearMap(randn(Ny, Nx))
Σϵ = LinearMap(PDiagMat(0.1 .+ rand(Ny)))
Dθ = LinearMap(PDiagMat(0.1 .+ rand(Ns)))
Σx = LinearMap(PDMat((x->x*x')(0.1*randn(Nx, Nx))+ 1.0*I))
S = LinearMap(PolyAnnil)

LoadError: UndefVarError: `PA` not defined

In [5]:
sys = ObsConstraintSystem(H, S, Dθ, Σϵ, Σx)

LoadError: UndefVarError: `S` not defined

In [6]:
sys_mat = zeros(Ny+Ns, Ny+Ns)

for i=1:Ny+Ns
    ei = zeros(Ny+Ns)
    ei[i] = 1.0
    sys_mat[:,i] = sys*ei
end

LoadError: UndefVarError: `sys` not defined

In [12]:
isposdef(Symmetric(sys_mat))

false

In [ ]:
sys.C

In [165]:
@time sys_op = FunctionMap{Float64,false}(x->sys*x, Ny+Ns; issymmetric=true, isposdef=true)

  0.005109 seconds (3.49 k allocations: 264.586 KiB, 92.30% compilation time)


130×130 FunctionMap{Float64,false}(#63; issymmetric=true, ishermitian=true, isposdef=true)

In [6]:
@time sys_op = FunctionMap{Float64,true}((y,x)->mul!(y, sys, x), Ny+Ns; issymmetric=true, isposdef=true)

  0.008216 seconds (14.28 k allocations: 1005.227 KiB, 92.18% compilation time)


130×130 FunctionMap{Float64,true}(#3; issymmetric=true, ishermitian=true, isposdef=true)

In [7]:
test = randn(Ny+Ns);
# test = ObsConstraintVector(randn(Ny), randn(Ns))

In [9]:
sys*test

LoadError: UndefVarError: `Ny` not defined

In [8]:
@time out = cg(sys_op, test; log = true)

LoadError: MethodError: no method matching mul!(::Vector{Float64}, ::ObsConstraintSystem, ::Vector{Float64}, ::Bool, ::Bool)

[0mClosest candidates are:
[0m  mul!([91m::ChainRulesCore.AbstractThunk[39m, ::Any, ::Any, ::Any, ::Any)
[0m[90m   @[39m [36mChainRulesCore[39m [90m~/.julia/packages/ChainRulesCore/zgT0R/src/tangent_types/[39m[90m[4mthunks.jl:99[24m[39m
[0m  mul!(::Any, [91m::ChainRulesCore.AbstractThunk[39m, ::Any, ::Any, ::Any)
[0m[90m   @[39m [36mChainRulesCore[39m [90m~/.julia/packages/ChainRulesCore/zgT0R/src/tangent_types/[39m[90m[4mthunks.jl:109[24m[39m
[0m  mul!(::Any, ::Any, [91m::ChainRulesCore.AbstractThunk[39m, ::Any, ::Any)
[0m[90m   @[39m [36mChainRulesCore[39m [90m~/.julia/packages/ChainRulesCore/zgT0R/src/tangent_types/[39m[90m[4mthunks.jl:110[24m[39m
[0m  ...


In [185]:
out

([0.0582400007442546, -0.06799136677872465, 0.006613694987607053, -0.3352344789037528, -0.16402890783652416, 0.12850517371105208, 0.023337062306841357, 0.014403122065477593, -0.11677822070149914, 0.06277727066535782  …  2.009235246908877, 1.4786102696103145, -0.8328221987627219, 0.03801512682955333, 0.45371557188236344, 0.6356738269348883, 0.4216776779055158, -0.6828494464585101, -0.7828993811851154, 0.566354519192026], Not converged after 130 iterations.)

In [186]:
sys_mat\test

130-element Vector{Float64}:
  0.05824002949404186
 -0.06799168280202558
  0.0066136717717618036
 -0.3352343644979481
 -0.16402866048783005
  0.12850497364039953
  0.023336956305048964
  0.01440234586566691
 -0.11677808608882748
  0.06277802999915877
 -0.06085863311124671
 -0.07048781907857132
 -0.13426897835608492
  ⋮
  1.1544917332585987
 -0.4417753639837578
  2.0092372311898767
  1.4786078153832516
 -0.8328199367597025
  0.03801327011871345
  0.45371658589223357
  0.6356749620625938
  0.42168040597632656
 -0.6828496456881498
 -0.7828949829222106
  0.5663548171127829

In [188]:
norm(sys_op*out[1] - test)#/norm(test)

0.00012354750507119743

In [137]:
norm(sys_mat\test - out[1])/norm(out[1])

0.0003997692205328466

In [25]:
sys*test

130-element Vector{Float64}:
 -15871.169236804346
 -16270.609480762432
 -22583.124405459355
   1318.7233179613522
 -28094.031818515818
   1382.5274400607825
  17143.18961157868
   8817.683007503783
  -4826.80048720641
   8175.059155101545
  26191.76750114389
  16332.72991787982
  44610.1172068557
      ⋮
     57.18127102022258
   1386.8808866634008
    -42.87407601831049
     -4.112728766230939
     -2.3470833893002867
   -662.9513821313901
    -37.0507273066059
  -2228.2641578466164
  -1577.6676769503188
    209.35693207745408
    -63.36499427714507
    -45.76282215302213